### Transformerの予測結果を重み付けして店舗ごとの予測結果として出力する

#### SET UP

In [1]:
import numpy as np
import pandas as pd

from Feature_Engineering import F_Engineering

#### データのインポート

In [2]:
df_train = pd.read_csv("./data/sales_train.csv")
df_test = pd.read_csv("./data/test.csv")

#### 店舗別アイテム別のデータ作成

In [3]:
# データのグルーピング
df_train_tuple = df_train.groupby(["shop_id", "item_id"]).sum()
df_train_tuple

/var/folders/6x/0whm9kbs4bg6p6xq6w7vsm0r0000gn/T/ipykernel_62492/680833413.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_train_tuple = df_train.groupby(["shop_id", "item_id"]).sum()


date_block_num  item_price  item_cnt_day
shop_id item_id                                          
0       30                    9      2385.0          31.0
        31                    7      3038.0          11.0
        32                    7      2431.0          16.0
        33                    3      2082.0           6.0
        35                   11      2964.0          15.0
...                         ...         ...           ...
59      22154                 0       999.0           1.0
        22155                 6       149.0           1.0
        22162               415      6234.0          16.0
        22164               160      4344.0           6.0
        22167                48      1196.0           4.0

[424124 rows x 3 columns]

In [4]:
# インデックスの整理
df_train_tuple = df_train_tuple.reset_index()
df_train_tuple

,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,0,30,9,2385.0,31.0
1,0,31,7,3038.0,11.0
2,0,32,7,2431.0,16.0
3,0,33,3,2082.0,6.0
4,0,35,11,2964.0,15.0
...,...,...,...,...,...
424119,59,22154,0,999.0,1.0
424120,59,22155,6,149.0,1.0
424121,59,22162,415,6234.0,16.0
424122,59,22164,160,4344.0,6.0


In [5]:
# 不要データのドロップ
df_train_tuple = df_train_tuple.drop(["date_block_num", "item_price"], axis = 1)
df_train_tuple

,shop_id,item_id,item_cnt_day
0,0,30,31.0
1,0,31,11.0
2,0,32,16.0
3,0,33,6.0
4,0,35,15.0
...,...,...,...
424119,59,22154,1.0
424120,59,22155,1.0
424121,59,22162,16.0
424122,59,22164,6.0


#### 重み付け前の均等割を実施

In [6]:
TF_forecast = 81353

df_test["item_cnt_month"] = TF_forecast * len(df_test) / len(df_train_tuple) / len(df_test)

df_test

,ID,shop_id,item_id,item_cnt_month
0,0,5,5037,0.191814
1,1,5,5320,0.191814
2,2,5,5233,0.191814
3,3,5,5232,0.191814
4,4,5,5268,0.191814
...,...,...,...,...
214195,214195,45,18454,0.191814
214196,214196,45,16188,0.191814
214197,214197,45,15757,0.191814
214198,214198,45,19648,0.191814


#### お重み付けを実施

In [7]:
# 重みの計算を実行
fe = F_Engineering(train = df_train, test = df_test)
w = fe.output_w()

/Users/udai/Predict-Future-Sales/udai_notebook/Feature_Engineering.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  shop_total = self.train.groupby("shop_id").sum()


In [ ]:
df_test_w = w

In [ ]:
# 商品合計の計算
item_total = np.sum(df_test_w.item_cnt_month)
item_total

46238.52175307223

In [ ]:
# 重みをつけたIDごとの商品数を出力する

## まずは、重み付加後の結果を格納する先を用意
df_test_w["item_cnt_month_w"] = 0

In [ ]:
## for文で、データ点分の重み付加計算を実行する

for i in range(df_test_w.shape[0]):
    df_test_w.item_cnt_month_w[i] = df_test_w.W[i] * item_total

/var/folders/6x/0whm9kbs4bg6p6xq6w7vsm0r0000gn/T/ipykernel_11853/3282009963.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_w.item_cnt_month_w[i] = df_test_w.W[i] * item_total


#### submissionの作成

In [ ]:
df_test_w

,ID,shop_id,item_id,item_cnt_month,W,item_cnt_month_w
0,0,5,5037,0.215866,0.000003,0.129647
1,1,5,5320,0.215866,0.000003,0.129647
2,2,5,5233,0.215866,0.000003,0.129647
3,3,5,5232,0.215866,0.000003,0.129647
4,4,5,5268,0.215866,0.000003,0.129647
...,...,...,...,...,...,...
214195,214195,45,18454,0.215866,0.000003,0.127018
214196,214196,45,16188,0.215866,0.000003,0.127018
214197,214197,45,15757,0.215866,0.000003,0.127018
214198,214198,45,19648,0.215866,0.000003,0.127018


In [ ]:
# 不要な列の削除
submission = df_test_w.drop(["shop_id", "item_id", "item_cnt_month", "W"], axis = 1)

# 列名の修正
submission.rename(columns = {"item_cnt_month_w" : "item_cnt_month"}, inplace = True)

# CSV出力
submission.to_csv("./submission/submission_4.csv", index = False)